<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Real-Time-Audio-Intercommunicator-(RTAI)" data-toc-modified-id="Real-Time-Audio-Intercommunicator-(RTAI)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span><a>Real Time Audio Intercommunicator (RTAI)</a></a></span><ul class="toc-item"><li><span><a href="#The-sounddevice.Stream-class" data-toc-modified-id="The-sounddevice.Stream-class-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>The <a href="https://python-sounddevice.readthedocs.io/en/0.3.15/api/streams.html" target="_blank"><code>sounddevice.Stream</code></a> class</a></span></li><li><span><a href="#Transmitting-live-data-through-the-Internet" data-toc-modified-id="Transmitting-live-data-through-the-Internet-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Transmitting live data through the Internet</a></span></li><li><span><a href="#Some-socket-stuff-in-Python" data-toc-modified-id="Some-socket-stuff-in-Python-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Some <a href="https://docs.python.org/3/library/socket.html" target="_blank"><code>socket</code></a> stuff in Python</a></span><ul class="toc-item"><li><span><a href="#Array-declarations" data-toc-modified-id="Array-declarations-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Array declarations</a></span></li><li><span><a href="#NumPy's-types" data-toc-modified-id="NumPy's-types-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>NumPy's types</a></span></li></ul></li></ul></li></ul></div>

## Transmitting live data through the Internet
Basically, when we want to transmit data between networked processes, two options are available:

1. [TCP](https://en.wikipedia.org/wiki/Transmission_Control_Protocol): A "reliable" transport protocol with possiblely high transmission latencies. Reliability means that the operating systems will *try* to solve the transmission errors, if they happen, by using [ARQ](https://en.wikipedia.org/wiki/Automatic_repeat_request).
2. [UDP](https://en.wikipedia.org/wiki/User_Datagram_Protocol): An unreliable transport protocol with a minimal (best-effort) transmission latencies. Unreliability means that the operating system will not try to fix any transmission error, if they happen (including the re-ordering of the packets).

Because we are interested in minimizing the total latency as much as possible, we will use UDP.

## Some [`socket`](https://docs.python.org/3/library/socket.html) stuff in Python

In [92]:
import socket

LISTENING_PORT = 8001

class UDP_receiver():
    # We use a context manager (https://docs.python.org/3/reference/datamodel.html#context-managers).
    def __enter__(self):
        '''Create an UDP socket and listen to it.'''
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        print("socket created")
        self.sock.bind(('', LISTENING_PORT))
        print(f"listening at {self.sock.getsockname()} ... ")
        return self

    def receive(self):
        '''Receive a datagram.'''
        (message, from_addr) = self.sock.recvfrom(1024) # Blocking operation, 1024 is the maximum expected payload size.
        print(f"received {message} from {from_addr}")
        return message
    
    def __exit__(self,ext_type,exc_value,traceback):
        '''Close the socket.'''
        self.sock.close()
        print("socket closed")

class UDP_sender():
    def __enter__(self):
        '''Create an UDP socket.'''
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        print("socket created")
        return self

    def send(self, message, destination):
        '''Send data.'''
        self.sock.sendto(message, destination)
        print(f"message {message} sent to destination {destination}")

    def __exit__(self,ext_type,exc_value,traceback):
        '''Close the socket.'''
        self.sock.close()
        print("socket closed")

def wait_for_a_message():
    with UDP_receiver() as receiver:
        message = receiver.receive().decode("utf-8")
        print(f"recived message = {message}")

def send_message():
    with UDP_sender() as sender:
        message = b"hello world!"
        destination = ('localhost', LISTENING_PORT)
        sender.send(message, destination)
        
import threading

threading.Thread(target=wait_for_a_message).start()
send_message()

socket created
listening at ('0.0.0.0', 8001) ... 
socket created
message b'hello world!' sent to destination ('localhost', 8001)received b'hello world!' from ('127.0.0.1', 51932)

recived message = hello world!
socket closed
socket closed


Notice that UDP is a datagram (independent packet) oriented protocol. The maximum packet size in UDP is 64 KB.

## And some [NumPy](https://numpy.org) stuff
Numpy extends lists in Python when we need to work with arrays of numbers, providing a higher performance (throught [vectorization](https://en.wikipedia.org/wiki/Array_programming)) and [functionality](https://numpy.org/doc/stable/reference/index.html).

In [2]:
try:
    import numpy as np
except ModuleNotFoundError:
    import os
    os.system("pip3 install numpy --user")
    import numpy as np

### Array declarations

In [5]:
l = [1, 2]
l

[1, 2]

In [6]:
type(l)

list

In [4]:
A = np.array(l) # Use "array", not "ndarray"
A

array([1, 2])

In [7]:
type(A)

numpy.ndarray

In [9]:
A[0]

1

In [12]:
Z = np.zeros((3,2,4))
Z

array([[[0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.]]])

### NumPy's data [types](https://numpy.org/devdocs/user/basics.types.html)

In [8]:
type(A[0])

numpy.int64

In [10]:
A = np.array(l, dtype=np.float32)
A

array([1., 2.], dtype=float32)

In [11]:
type(A[0])

numpy.float32

In [13]:
Z = np.zeros((3,2,4), dtype=np.uint8)
Z

array([[[0, 0, 0, 0],
        [0, 0, 0, 0]],

       [[0, 0, 0, 0],
        [0, 0, 0, 0]],

       [[0, 0, 0, 0],
        [0, 0, 0, 0]]], dtype=uint8)

In [14]:
type(Z[1,1,1])

numpy.uint8

### Slicing

In [15]:
A = np.arange(10)**3
A

array([  0,   1,   8,  27,  64, 125, 216, 343, 512, 729])

In [16]:
A[2] # Indexing

8

In [17]:
A[2:4] # Indexing using slicing

array([ 8, 27])

In [18]:
A[2:] # Slicing

array([  8,  27,  64, 125, 216, 343, 512, 729])